# top

In [ ]:
import scipp as sc
import numpy as np
def make_array():
    data = sc.array(dims=['xx','yy','zz'], unit='', values=10*np.random.rand(4,40,4))
    data = sc.array(dims=['xx','yy','zz'], unit='K', values=np.sin(0.1*np.arange(4*4*40)).reshape(4,40,4), variances=0.01*np.ones((4,40,4)))
    #data = sc.array(dims=['xx','yy','zz'], unit='counts', values=np.sin(0.1*np.arange(4*4*40)).reshape(4,40,4), variances=0.01*np.ones((4,40,4)))
    da = sc.DataArray(data=data)
    line = da['zz',0]['yy', 0].copy()
    da.coords['yy'] = sc.linspace(dim='yy', unit='km', start=1.1, stop=5.2, num=40)
    da.coords['xx'] = sc.linspace(dim='xx', unit='mm', start=1.1, stop=5.2, num=5)
    da.coords['xx2'] = sc.linspace(dim='xx', unit='m', start=1.1, stop=55.2, num=5)
    da.masks['mask'] = da.coords['xx']['xx',1:] > 5.0 * sc.Unit('mm')
    return da

In [ ]:
import scipp as sc
import numpy as np
da = make_array()
da

# 1d plotting

In [ ]:
da['zz',0].plot(projection='1d', vmin=-2*da.unit, vmax=2*da.unit)
#da['zz',0].plot(projection='1d')
#da['yy',0].plot(projection='1d')

In [ ]:
#da.plot(projection='1d', vmin=-2, vmax=2)
da.plot(projection='1d', vmin=-2*da.unit, vmax=2*da.unit)
#da.plot(projection='1d')

## 1d plot of non-counts

Check that normalization not messed up, plot should show a constant line at $1~\text{K}$:

In [ ]:
import scipp as sc
da = sc.DataArray(data=sc.ones(dims=['x'], shape=[100], unit='K'),
                  coords={'x':sc.geomspace(dim='x', start=1, stop=1e6, num=100)})
da.plot()

# Datetime coords

In [ ]:
da = make_array()
# TODO zoom exception
#da.coords['xx'] = sc.scalar(np.datetime64("now")) + sc.linspace(dim='xx', unit='s', dtype='int64', start=1, stop=3, num=5)
da.coords['xx'] = sc.scalar(np.datetime64("now")) + sc.linspace(dim='xx', unit='s', dtype='int64', start=1, stop=5000, num=5)
da['zz',0].plot(projection='1d')

In [ ]:
da['zz',0]['yy',0].plot()

In [ ]:
da.plot()

In [ ]:
da['zz', 0].plot()

In [ ]:
dt = '2021-07-06T07:16:10'
len(dt)

## 2d plot of 3d data with slider

- Custom labels for `xx` should be used for:
  - axis tics
  - button for dim selection
  - slider range display
  - line label if line kept in profile plot

In [ ]:
#da.plot(axes={'x':'xx2'})
da = make_array()
da.plot(labels={'xx':'xx2'}, errorbars=False)

## 2d plot of 3d counts data with slider

- Profile readout must be consistent through zoom levels

In [ ]:
#da.plot(axes={'x':'xx2'})
da = make_array()
da.unit = 'counts'
da.plot(labels={'xx':'xx2'}, errorbars=False)

In [ ]:
#sc.plot({'a':da['zz',0]['yy', 1], 'b':line})

In [ ]:
sc.plot({'a':da['zz',0], 'b':da['zz',1]}, projection='1d')

In [ ]:
da.data.plot()

In [ ]:
da4d = sc.values(sc.concatenate(da, da*sc.scalar(2.0), 'time'))
da4d.plot()

## `redraw`

In [ ]:
da1d = da['xx',0]['zz',0].copy()
plot = da1d.plot()
plot.show()

Running this cell should flip data (vertical axis) in plot above:

In [ ]:
da1d.data *= -1.0
plot.redraw()

In [ ]:
da2d = da['xx',0].copy()
plot = da2d.plot()
plot.show()

Running this cell should flip data (color axis) in plot above:

In [ ]:
da2d.data *= -1.0
plot.redraw()

## Multi-dimensional coord

In [ ]:
da_multi = da.copy()
da_multi.coords['zz'] = sc.array(dims=['yy', 'zz'], unit='um', values=np.arange(160).reshape(40,4))
da_multi['xx',0].plot()

In [ ]:
da_multi.plot()

In [ ]:
try:
    da_multi.transpose().plot()
except sc.DimensionError as e:
    print(e)

# Binned data

In [ ]:
import scipp as sc
import numpy as np
N = int(80000)
values = np.random.rand(N)
data = sc.DataArray(
    data=sc.Variable(dims=['event'], values=values, unit=''),
    coords={
        'x':sc.Variable(dims=['event'], values=np.random.rand(N)),
        'y':sc.Variable(dims=['event'], values=np.random.rand(N)),
        'z':sc.Variable(dims=['event'], values=np.random.rand(N))
    })
binned = sc.bin(data, edges=[sc.Variable(dims=['x'], values=np.linspace(0.0,1.0,num=10)),
                             sc.Variable(dims=['y'], values=np.linspace(0.0,1.0,num=4)),
                             sc.Variable(dims=['z'], values=np.linspace(0.0,1.0,num=4))])
binned.plot(resolution=100)

In [ ]:
binned['x', 0].plot(resolution=100)

In [ ]:
binned['x', 0].plot(projection='1d', resolution=100)

In [ ]:
binned['x', 0]['y', 0].plot()

# 3d plotting

In [ ]:
import scipp as sc
import numpy as np
N = 10
M = 40
L = 30
K = 20
xx = np.arange(3, N, dtype=np.float64)
yy = np.arange(4, M, dtype=np.float64)
zz = np.arange(5, L, dtype=np.float64)
qq = np.arange(6, K, dtype=np.float64)
x, y, z, q = np.meshgrid(xx, yy, zz, qq, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2  + ((q-d)/b)**2)
a = np.sin(r)
d = sc.Dataset()
da = sc.DataArray(data=sc.Variable(dims=['xx', 'yy', 'zz', 'Q_x'], values=a, unit='K'))
da.coords['xx'] = sc.Variable(dims=['xx'], values=xx)
da.coords['yy'] = sc.Variable(dims=['yy'], values=yy, unit='m')
da.coords['zz'] = sc.Variable(dims=['zz'], values=zz)
da.coords['Q_x'] = sc.Variable(dims=['Q_x'], values=qq)
sc.plot(da, projection='3d', aspect='auto')

In [ ]:
import scipp as sc
import numpy as np
N = 1000
M = 100
theta = np.random.random(N) * np.pi
phi = np.random.random(N) * 2.0 * np.pi
r = 10.0 + (np.random.random(N) - 0.5)
x = r * np.sin(theta) * np.sin(phi)
y = r * np.sin(theta) * np.cos(phi)
z = r * np.cos(theta)

a = np.arange(M*N).reshape([M, N]) * np.sin(y)
pos = sc.vectors(dims=['xyz'], unit='m', values=np.array([x, y, z]).T)
da = sc.DataArray(
    data=sc.array(dims=['time', 'xyz'], values=a, unit='K'),
    #masks={
    #    'mask':pos.fields.x < 0.0 * sc.Unit('m')},
    coords={
        'xyz':pos,
        'time':sc.array(dims=['time'], values=np.arange(M).astype(np.float))})
# labels is hack so model does not drop our pos coord
# need panel so slices get defined in controller.update_axes
#da['time',0].plot(projection="3d", positions='xyz', labels={'xyz':'xyz'})
da.plot(projection="3d", positions='xyz')